In [5]:
import numpy as np
np.random.seed(19906)
import pandas as pd

选择作为验证集的司机ID

In [3]:
drivers_pd = pd.read_csv("data/drivers_img_nop081_list.csv")
imgs_pd = drivers_pd["img"]
class_pd = drivers_pd["classname"]
subject_pd = drivers_pd["subject"]
# choices = np.random.choice(drivers_pd["subject"].drop_duplicates(), 2)
choices = ['p041', 'p056']
print("选择作为验证集的司机ID:", choices)

选择作为验证集的司机ID: ['p041', 'p056']


按选择的司机ID分割训练集和验证集

In [6]:
val_index = []
for choice in choices:
    val_index.extend(subject_pd[subject_pd == choice].index.tolist())
    
test_mask = np.zeros(np.alen(subject_pd), dtype=np.bool)
for val_i in val_index:
    test_mask[val_i] = True
    
train_index = subject_pd[np.logical_not(test_mask)].index
print("分割的训练集数量:", np.alen(train_index), "，验证集数量:", np.alen(val_index))

分割的训练集数量: 20925 ，验证集数量: 1399


读取被数据增强处理的图片

In [7]:
drivers_aug_pd = pd.read_csv("data/drivers_img_aug_list.csv")
imgs_aug_pd = drivers_aug_pd["img"]
class_aug_pd = drivers_aug_pd["classname"]
subject_aug_pd = drivers_aug_pd["subject"]

In [8]:
exclude_index = []
for choice in choices:
    exclude_index.extend(subject_aug_pd[subject_aug_pd == choice].index.tolist())
test_aug_mask = np.zeros(np.alen(subject_aug_pd), dtype=np.bool)
for val_i in exclude_index:
    test_aug_mask[val_i] = True
    
train_aug_index = subject_aug_pd[np.logical_not(test_aug_mask)].index
print("从增强数据集中分割的训练集数量:", np.alen(train_aug_index))

从增强数据集中分割的训练集数量: 20925


创建图像数据处理目录

In [10]:
import os
import shutil

def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)
    
train_dir = "data/imgs/train2"
val_dir = "data/imgs/val2"
origin_test_dir = "data/imgs/test"
test_dir = "data/imgs/test1"
saved_weights = "saved_weights"
if not os.path.exists(saved_weights):
    os.mkdir(saved_weights)
    
# 因为加载测试集时目录中也需要有子目录，将data/imgs/test目录软链接到data/imgs/test1/test
if not os.path.exists(test_dir):
    os.mkdir(test_dir)
    os.symlink('../test', test_dir+"/test")

# 在新的训练或验证集目录中为图片创建到原位置的链接
def link_imgs(target_dir, X, y):
    for img_name, target in zip(X, y):
        symlink_dir = os.path.join(target_dir, target)
        if not os.path.exists(symlink_dir):
            os.mkdir(symlink_dir)
        os.symlink('../../train/'+target+'/'+img_name, symlink_dir+'/'+img_name)
        
def link_aug_imgs(target_dir, X, y):
    for img_name, target in zip(X, y):
        symlink_dir = os.path.join(target_dir, target)
        if not os.path.exists(symlink_dir):
            os.mkdir(symlink_dir)
        os.symlink('../../train_aug/'+target+'/'+img_name, symlink_dir+'/'+img_name)

数据准备

In [13]:
# 删除上次分离出的训练集和验证集文件，并重新创建目录
rmrf_mkdir(train_dir)
rmrf_mkdir(val_dir)

X_train, X_val = imgs_pd[train_index], imgs_pd[val_index]
y_train, y_val = class_pd[train_index], class_pd[val_index]

# 链接训练集到新的目录中
link_imgs(train_dir, X_train, y_train)

# 链接增强训练集到新的目录中
X_aug_train, y_aug_train = imgs_aug_pd[train_aug_index], class_aug_pd[train_aug_index]
link_aug_imgs(train_dir, X_aug_train, y_aug_train)

# 链接验证集到新的目录中
link_imgs(val_dir, X_val, y_val)

print("split valid data done!")

split valid data done!
